In [3]:
# ----------------------------------------
# 📦 Step 1: Import required libraries
# ----------------------------------------
from google_play_scraper import reviews, Sort       # To scrape Google Play app reviews
import pandas as pd                                 # For data manipulation and analysis
from datetime import datetime                       # To normalize and format dates
import os                                           # For creating folders and managing paths
import emoji
import re

from deep_translator import GoogleTranslator  # For translation
from langdetect import detect, LangDetectException  # For detecting language

# ----------------------------------------
# 🏦 Step 2: Define target apps and corresponding bank names
# ----------------------------------------
# This dictionary maps each app's package name (used for scraping)
# to a human-readable bank name for clarity in the dataset
apps = {
    'com.combanketh.mobilebanking': 'CBE',       # Commercial Bank of Ethiopia
    'com.boa.boaMobileBanking': 'BOA',           # Bank of Abyssinia
    'com.dashen.dashensuperapp': 'Dashen'        # Dashen Bank
}

# ----------------------------------------
# 📁 Step 3: Ensure directory for raw data exists
# ----------------------------------------
# Create a folder structure: data/raw
output_dir = os.path.join('../data/raw')          # Set the desired path
#os.makedirs(output_dir, exist_ok=True)            # Create the folder if it doesn't exist

# ----------------------------------------
# 🧺 Step 4: Initialize a list to store all reviews
# ----------------------------------------
# We'll append each app's reviews as a DataFrame to this list and combine them later
all_reviews = []



def is_only_emoji(text):
    if pd.isna(text):
        return False
    text = text.strip()
    text = re.sub(r'[\uFE0F\u200D]', '', text)  # remove variation selectors, zero width joiners
    emoji_pattern = re.compile(
        r'^[\U0001F600-\U0001F64F'  # emoticons
        r'\U0001F300-\U0001F5FF'    # symbols & pictographs
        r'\U0001F680-\U0001F6FF'    # transport & map symbols
        r'\U0001F1E0-\U0001F1FF'    # flags (iOS)
        r'\U00002702-\U000027B0'    # dingbats
        r'\U000024C2-\U0001F251]+$' # enclosed characters
    )
    return bool(emoji_pattern.fullmatch(text))




# ---------------------------------------------------------
# Function to detect the language of a given text
# ---------------------------------------------------------
def detect_language(text):
    try:
        # langdetect returns ISO 639-1 language codes like 'en', 'am', 'fr', etc.
        return detect(text)
    except LangDetectException:
        # If language detection fails (e.g., empty or strange text), return 'unknown'
        return "unknown"

# ---------------------------------------------------------
# Function to translate text ONLY if it's not in English
# ---------------------------------------------------------
def smart_translate(text):
    # Skip translation if text is missing or not a valid string
    if pd.isna(text) or not isinstance(text, str):
        return text
    
    # Step 1: Detect language
    lang = detect_language(text)
    
    # Step 2: If the language is not English, translate it
    if lang != 'en':
        try:
            # Use GoogleTranslator from deep_translator (auto-detects source language)
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        except Exception as e:
            # If translation fails (e.g., API issue), print error and return original
            print(f"Translation failed for: {text} — Error: {e}")
            return text
    else:
        # If already in English, return it as-is
        return text















# ----------------------------------------
# 🔁 Step 5: Loop through each app and scrape reviews
# ----------------------------------------
for package_name, bank_name in apps.items():
    print(f"📥 Scraping reviews for {bank_name}...")

    # Use google_play_scraper to fetch up to ~500 recent reviews
    reviews_list, _ = reviews(
        package_name,               # App identifier
        lang='en',                  # Language: English
        country='et',               # Country: US (change to 'et' for Ethiopia if needed)
        sort=Sort.NEWEST,           # Get the most recent reviews
        count=500                   # Number of reviews to fetch
    )

    # Convert list of dictionaries to a DataFrame
    df = pd.DataFrame(reviews_list)

    # Add custom columns for 'bank' and 'source' (Google Play)
    df['bank'] = bank_name
    df['source'] = 'Google Play'

    # Append this bank's reviews to the main list
    all_reviews.append(df)

# ----------------------------------------
# 📊 Step 6: Combine all banks' data into a single DataFrame
# ----------------------------------------
combined_df = pd.concat(all_reviews, ignore_index=True)

# ----------------------------------------
# 🧹 Step 7: Preprocess the data
# ----------------------------------------

# 7.1 Remove duplicate rows based on content (e.g., review content + userName)
combined_df.drop_duplicates(subset=['content', 'userName'], inplace=True)

# 7.2 Handle missing values — remove rows where 'content' or 'score' is missing
combined_df.dropna(subset=['content', 'score'], inplace=True)

# 7.3 Normalize the date format — convert 'at' column to YYYY-MM-DD
combined_df['date'] = pd.to_datetime(combined_df['at']).dt.date

# ----------------------------------------
# 📦 Step 8: Select and rename the required columns
# ----------------------------------------
final_df = combined_df[['content', 'score', 'date', 'bank', 'source']]
final_df.columns = ['review', 'rating', 'date', 'bank', 'source']  # Rename columns

final_df = final_df[~final_df['review'].apply(is_only_emoji)]

# ---------------------------------------------------------
# Apply the function to your DataFrame's 'review' column
# ---------------------------------------------------------
# This line goes through every review and applies the smart_translate logic
final_df['review'] = final_df['review'].apply(smart_translate)
# ----------------------------------------
# 💾 Step 9: Save cleaned data to CSV
# ----------------------------------------
csv_path = os.path.join(output_dir, 'banks_review_cleaned.csv')
final_df.to_csv(csv_path, index=False)

# ✅ Done
print(f"✅ Scraping and cleaning complete! Data saved to: {csv_path}")


📥 Scraping reviews for CBE...
📥 Scraping reviews for BOA...
📥 Scraping reviews for Dashen...
Translation failed for: good — Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=auto&q=good (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FC9A71C190>, 'Connection to translate.google.com timed out. (connect timeout=None)'))
Translation failed for: Engida Kebede Fetera — Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=auto&q=Engida+Kebede+Fetera (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FC9A71C7D0>, 'Connection to translate.google.com timed out. (connect timeout=None)'))
Translation failed for: 🤬🤬🤬🤬 network 🛜 — Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=auto&q=%F0%9F%A4%AC%F0%9F%A4%AC%F0%9F%A4%AC%F0%9F%A4%AC+network+%F0%9F%9B%9C

In [6]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('../data/raw/banks_review_cleaned.csv')

# Count the number of reviews for each bank
bank_counts = df['bank'].value_counts()

# Print the counts
print("📊 Number of reviews per bank:")
print(bank_counts)
print(df.isnull().sum())



📊 Number of reviews per bank:
bank
CBE       495
BOA       488
Dashen    444
Name: count, dtype: int64
review    0
rating    0
date      0
bank      0
source    0
dtype: int64
